## Prediction

In [4]:
# Import Libraries
import numpy as np
import tensorflow as tf

from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [9]:
# Load Dataset and word index
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value  in word_index.items()}

In [10]:
# Load pretrained model
model = load_model("../model/rnn_imdb.h5")
model.summary() # Check model summary

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (32, 500, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (32, 128)              │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (32, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [20]:
# Helper function to decode review
# Decoded function to convert encoded reviews back to text
def decode_review(encoded_review):
  '''
  Decodes a sequence of integers back to a human-readable string.
  Args:
    encoded_review (list of int): The encoded review as a list of integers.
    reverse_word_index (dict): A dictionary mapping integers to words.
  '''
  return " ".join([reverse_word_index.get(i - 3, '?') for i in encoded_review])


# Preprocess function to convert reviews to a format suitable for the model
def preprocess_text(text):
  '''
  Converts a text review into a padded sequence of integers.
  Args:
    text (str): The review text to preprocess.
    word_index (dict): A dictionary mapping words to their integer indices.
  '''
  words = text.lower().split()
  encoded_review = [word_index.get(word, 2) + 3 for word in words]
  padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
  return padded_review

In [21]:
# Prediction function
def predict_sentiment(review):
  '''
  Predicts the sentiment of a given review.
  Args:
    review (str): The review text to analyze.
  '''
  preprocessed = preprocess_text(review)
  prediction = model.predict(preprocessed)
  sentiment = "Positive" if prediction[0][0] > 0.5 else "Negative"
  return sentiment, prediction[0][0]

In [23]:
# Example prediction
example_review = "This movie was fantastic! I loved the plot and the acting was superb."
sentiment, prediction = predict_sentiment(example_review)

# Output the prediction
print(f"Review : {example_review}")
print(f"Sentiment: {sentiment}")
print(f"Prediction Score: {prediction:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Review : This movie was fantastic! I loved the plot and the acting was superb.
Sentiment: Positive
Prediction Score: 0.9854
